# DP02 Survey Property Tutorial

By Eli Rykoff

This should be run on a large instance (for the memory) with w_2022_22 (recommended)

In [ ]:
# Standard imports, plus skyproj for visualization
from lsst.daf.butler import Butler
import numpy as np
import healsparse as hsp
import healpy as hp
import matplotlib.pyplot as plt
import skyproj

# This will allow us to have interactive plots
%matplotlib widget

In [ ]:
# This is where the survey property maps live
config = 'dp02'
collections = '2.2i/runs/DP0.2'

config = 's3://butler-us-central1-panda-dev/dc2/butler-external.yaml'
conllections = "2.2i/runs/DP0.2/v23_0_2/PREOPS-905/step7"

butler = Butler(config, collections=collections)

In [ ]:
# Let's determine which property maps are available for the survey
for dtype in butler.registry.queryDatasetTypes(expression="*consolidated_map*"):
    print(dtype.name)

The meaning of these types is:

* deepCoadd_exposure_time_consolidated_map_sum: Total exposure time (seconds)
* deepCoadd_psf_maglim_consolidated_map_weighted_mean: PSF Flux 5-sigma magnitude limit (AB)
* deepCoadd_sky_background_consolidated_map_weighted_mean: Weighted mean of sky background of images input to coadd (ADU)
* deepCoadd_sky_noise_consolidated_map_weighted_mean: Weighted mean of sky noise of images input to coadd (ADU)
* deepCoadd_psf_size_consolidated_map_weighted_mean: Weighted mean of psf size of images input to coadd (pixels)
* deepCoadd_psf_e1_consolidated_map_weighted_mean: Weighted mean of psf e1 of images input to coadd
* deepCoadd_psf_e2_consolidated_map_weighted_mean: Weighted mean of psf e2 of images input to coadd
* deepCoadd_dcr_dra_consolidated_map_weighted_mean: Average effect of differential chromatic refraction (DCR) in RA direction
* deepCoadd_dcr_ddec_consolidated_map_weighted_mean: Average effect of differential chromatic refraction (DCR) in Dec direction
* deepCoadd_dcr_e1_consolidated_map_weighted_mean: Average effect of differential chromatic refraction (DCR) on psf e1
* deepCoadd_dcr_e2_consolidated_map_weighted_mean: Average effect of differential chromatic refraction (DCR) on psf e2

Note that the DCR maps are proportionality maps; that is, the expected effect will be proportional to the value in the map with an arbitrary/empirically derived constant of proportionality.

In [ ]:
# In order to read a map, we need to specify the map name and a band.
hspmap = butler.get('deepCoadd_psf_maglim_consolidated_map_weighted_mean', band='i')

In [ ]:
# Use skyproj (https://skyproj.readthedocs.io/en/latest/) to plot the map
# The map will be interactive in notebook mode
fig, ax = plt.subplots(num=1, figsize=(10, 7))
# This creates a visualization using the McBryde-Thomas Flat Polar Quartic, centered at 65 deg longitude, 
#  appropriate for the DP0.2 footprint
sp = skyproj.McBrydeSkyproj(ax=ax, lon_0=65.0)
sp.draw_hspmap(hspmap)
sp.draw_colorbar(label='PSF Maglim (i-band)')
plt.show()

In [ ]:
# We can make another visualization with a new colormap to avoid the shallow edges
fig, ax = plt.subplots(num=2, figsize=(10, 7))
# This creates a visualization using the McBryde-Thomas Flat Polar Quartic, centered at 65 deg longitude, 
#  appropriate for the DP0.2 footprint
sp = skyproj.McBrydeSkyproj(ax=ax, lon_0=65.0)
sp.draw_hspmap(hspmap, vmin=25.8, vmax=26.3)
sp.draw_colorbar(label='PSF Maglim (i-band)')
plt.show()

Note that the above maps are interactive.  You can draw boxes and zoom in, etc.  As you move the mouse around the bottom label will show the longitude, latitude, and map value.  When you zoom in the default is that the colorbar will autoscale according to the data in the box.  If you want to turn this off you can specify `sp.set_autorescale(False)`

You can also zoom in specifically by specifying a longitude/latitude range, as below.

In [ ]:
# And we can zoom in initially as well.  I've changed the vmin/vmax to make it look better.
fig, ax = plt.subplots(num=2, figsize=(10, 7))
sp = skyproj.McBrydeSkyproj(ax=ax, lon_0=65.0)
sp.draw_hspmap(hspmap, vmin=26.0, vmax=26.3, lon_range=[55, 60], lat_range=[-40, -35])
sp.draw_colorbar(label='PSF Maglim (i-band)')
plt.show()

In [ ]:
# In order to get values out of the map, we use healsparse (https://healsparse.readthedocs.io/en/latest/)
# You can do any sort of lookup by position or (nest) pixel value.
values = hspmap.get_values_pos(np.array([55, 65]), np.array([-40, -30]))
print(values)